# Univariate Model-Based Selection

### Imports

In [1]:
import pandas as pd
import numpy as np
import pickle
import os
import gc
import warnings
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional
from tensorflow.keras.layers import Conv1D, Flatten, RepeatVector, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

warnings.filterwarnings("ignore")

Configuration

In [2]:
TARGET_VARIABLES = ['co2', 'gdp', 'primary_energy_consumption']
N_STEPS_IN = 5
N_STEPS_OUT = 3
VAL_SIZE = 0.15
TEST_SIZE = 0.15
G20_COUNTRIES = [
    'United States', 'China', 'Japan', 'Germany', 
    'United Kingdom', 'France', 'Italy', 'Canada',
    'Brazil', 'Russia', 'India', 'Australia', 
    'Mexico', 'Indonesia', 'Turkey', 'Saudi Arabia',
    'South Africa', 'Argentina', 'South Korea'
]

### Data Loads

In [3]:
def load_data(save_dir='data_export'):

    selected_features_path = os.path.join(save_dir, '02_results', 'selected_features.pkl')
    lag_path = os.path.join(save_dir, 'lag_df_1965.pkl')

    try:
        with open(selected_features_path, 'rb') as f:
            selected_features = pickle.load(f)
        print(f"Selected features loaded")

        lag_df = pd.read_pickle(lag_path)
        print(f"Time lag dataframe loaded")

        return selected_features, lag_df
    
    except FileNotFoundError as e:
        print(f"Error in loading {e}")
        return None, None

In [4]:
selected_features, lag_df = load_data()

Selected features loaded
Time lag dataframe loaded


### Utility Functions

In [5]:
def clear_memory():
    tf.keras.backend.clear_session()
    gc.collect()

In [6]:
# Root Mean Square Error
def rmse(y_pred, y_actual):
    return np.sqrt(mean_squared_error(y_actual, y_pred))

In [7]:
# Mean Absolute Scaled Error
def mase(y_actual, y_pred, period=1):

    mae_forecast = mean_absolute_error(y_actual, y_pred)
    
    # MAE of naive forecast
    naive_forecast = y_actual[:-period] if period > 0 else y_actual[:-1]
    actual_for_naive = y_actual[period:] if period > 0 else y_actual[1:]
    
    if len(naive_forecast) == 0:
        return np.nan
    
    mae_naive = mean_absolute_error(actual_for_naive, naive_forecast)
    
    if mae_naive == 0:
        return 0 if mae_forecast == 0 else np.inf
    
    return mae_forecast / mae_naive

In [8]:
# Checking stationary for ARIMA model
def check_stationarity(data, alpha=0.05):
    # P_value = result[1]
    result = adfuller(data)
    return result[1] < alpha, result[1]

In [9]:
# Making stationary if not with differencing
def make_stationary(data, max_diff=2):

    diff_order = 0
    curr_data = data.copy()

    for d in range(max_diff + 1):
        is_stationary, p_value = check_stationarity(curr_data)
        if is_stationary:
            return curr_data, d
        if d < max_diff:
            curr_data = np.diff(curr_data)
            diff_order += 1

    return curr_data, diff_order

In [10]:
# Inversing diff for differeced forecasts
def inv_diff(forecast, original_series, d):

    if d == 0:
        return forecast
    
    # Last values for inv diff
    last_values = []
    temp_series = original_series.copy()

    for i in range(d):
        last_values.append(temp_series[-1])
        if i < d - 1:
            temp_series = np.diff(temp_series)

    # Inversing
    result = forecast
    for i in range(d-1, -1, -1):
        result = np.cumsum(np.concatenate([[last_values[i]], result]))
        # Remove last_values[i]
        result = result[1:]

    return result

### Data Prep

In [11]:
# Preparing for data for ARIMA
def prepare_ARIMA_data(country_data, feature, target, n_steps_in=5):
    
    # (curr + lags)
    feature_cols = [feature]
    for i in range(1, n_steps_in):
        lag_col = f"{feature}_lag{i}"
        if lag_col in country_data.columns:
            feature_cols.append(lag_col)
    
    all_cols = feature_cols + [target] + ['year']
    data_subset = country_data[all_cols].dropna()
    
    # (n_samples, n_features)
    X = data_subset[feature_cols].values
    # (n_samples,) -> change into 1D array with ravel()
    y = data_subset[target].values.ravel()
    years = data_subset['year'].values

    print(f"ARIMA shape X:{X.shape}, y:{y.shape}")
    
    # Split for time series
    n_train = int(len(y) * (1 - TEST_SIZE - VAL_SIZE))
    n_val = int(len(y) * VAL_SIZE)
    
    X_train = X[:n_train]
    y_train = y[:n_train]
    X_val = X[n_train:n_train + n_val]
    y_val = y[n_train:n_train + n_val]
    X_test = X[n_train + n_val:]
    y_test = y[n_train + n_val:]

    test_years = years[n_train+n_val:]

    print(f"Test data length: {len(y_test)}, Test years length: {len(test_years)}")
    
    return X_train, y_train, X_val, y_val, X_test, y_test, test_years

In [12]:
# Preparing data for DL models
def prepare_uni_data(country_data, feature, target, n_steps_in=5):
    """
    Filtering out the data with the feature and its lags to predict target
    """

    feature_cols = [feature]
    for i in range(1, n_steps_in):
        lag_col = f"{feature}_lag{i}"
        if lag_col in country_data.columns:
            feature_cols.append(lag_col)
    
    # Subset data
    target_col = target
    all_cols = feature_cols + [target_col] + ['year']
    data_subset = country_data[all_cols].dropna()
    
    feature_data = []
    for col in feature_cols:
        feature_data.append(data_subset[col].values)

    X = np.column_stack(feature_data)
    y = data_subset[target_col].values
    years = data_subset['year'].values
    
    return X, y, years

In [13]:
def create_sequence(X, y, n_steps_out=3):
    """
    With prepared data, it creates n_steps_out number of sequences for multi-step prediction.
    [t-4, t-3, t-2, t-1, t] -> [t+1, t+2, t+3]
    """
    X_seq, y_seq = [], []

    for i in range(len(y) - n_steps_out):
        X_seq.append(X[i])

        y_output = []
        for j in range(1, n_steps_out+1):
            if i+j < len(y):
                y_output.append(y[i+j])

        if len(y_output) == n_steps_out:
            y_seq.append(y_output)
        else:
            # Remove last X when not enough y values
            X_seq.pop()

    return np.array(X_seq), np.array(y_seq)

In [14]:
def prepare_dl_data(X, y, years, n_steps_out=3, val_size=0.15, test_size=0.15):

    # Sequence
    X_seq, y_seq = create_sequence(X, y, n_steps_out)
    years_seq = years[:len(X_seq)]

    # Split data
    n_total = len(X_seq)
    n_train = int(n_total * (1 - test_size - val_size))
    n_val = int(n_total * val_size)

    X_train = X_seq[:n_train]
    y_train = y_seq[:n_train]
    X_val = X_seq[n_train:n_train + n_val]
    y_val = y_seq[n_train:n_train + n_val]
    X_test = X_seq[n_train + n_val:]
    y_test = y_seq[n_train + n_val:]
    test_years = years_seq[n_train + n_val:]

    # StandardScaler is better for future extrapolation wrt time series (2D)
    X_scaler = StandardScaler()
    y_scaler = StandardScaler()
    
    X_scaler.fit(X_train)
    y_scaler.fit(y_train)
    
    X_train_scaled = X_scaler.transform(X_train)
    y_train_scaled = y_scaler.transform(y_train)
    X_val_scaled = X_scaler.transform(X_val)
    y_val_scaled = y_scaler.transform(y_val)
    X_test_scaled = X_scaler.transform(X_test)
    
    # Reshape for LSTM (n_samples, timesteps, n_features)
    X_train_dl = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
    X_val_dl = X_val_scaled.reshape(X_val_scaled.shape[0], X_val_scaled.shape[1], 1)
    X_test_dl = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

    #print(f"Scaling range Train: {y_train_scaled.min():.4f} - {y_train_scaled.max():.4f}")

    return (X_train_dl, y_train_scaled, X_val_dl, y_val_scaled, X_test_dl, 
            y_test, y_scaler, test_years)

### Model builds

In [15]:
# Tuning ARIMA with AIC criterion
def tune_ARIMA_params(y_train, exog_train=None, max_p=4, max_d=2, max_q=3):
    """
    p (autoregressive order): num of lag obs to include
    d (differencing degree): num of time data different to make it stationary
    q (moving avg order): size of moving avg window
    Lower AIC -> better model fit
    """

    best_aic = np.inf
    best_params = None
    best_model = None

    # Check if it is stationary
    is_stationary, p_value = check_stationarity(y_train)

    if is_stationary:
        d_range = [0]
    else:
        _, d_needed = make_stationary(y_train, max_diff=2)
        print(f"    Not stationary p_value: {p_value:.4f}")
        d_range = [1]

    # Grid search
    for p in range(max_p + 1):
        for d in d_range:
            for q in range(max_q + 1):
                try:
                    model = ARIMA(y_train, exog=exog_train, order=(p, d, q))
                    fitted_model = model.fit()
                    aic = fitted_model.aic

                    if aic < best_aic:
                        best_aic = aic
                        best_params = (p, d, q)
                        best_model = fitted_model
                
                except:
                    continue

    return best_model, best_params, best_aic

In [16]:
# Simple LSTM model
def build_lstm(input_shape, n_steps_out, hidden=16):
    model = Sequential([
        LSTM(hidden, activation='relu', input_shape=input_shape, kernel_regularizer=l2(0.001)),
        Dense(n_steps_out)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

In [17]:
# Bi-directional LSTM
def build_bilstm(input_shape, n_steps_out, hidden=8):
    model = Sequential([
        Bidirectional(LSTM(hidden, activation='relu', kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001)),
                      input_shape=input_shape),
        Dense(n_steps_out)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

In [18]:
# Encoder-Decoder LSTM
def build_edlstm(input_shape, n_steps_out, hidden=8):
    model = Sequential([
        LSTM(hidden, activation='relu', input_shape=input_shape, kernel_regularizer=l2(0.001)),
        RepeatVector(n_steps_out),
        LSTM(hidden, activation='relu', return_sequences=True, kernel_regularizer=l2(0.001)),
        TimeDistributed(Dense(1))
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

In [19]:
# CNN model
def build_cnn(input_shape, n_steps_out, filters=32, hidden=8):
    model = Sequential([
        Conv1D(filters=filters, kernel_size=3, activation='relu', input_shape=input_shape, padding='same', kernel_regularizer=l2(0.001)),
        Flatten(),
        Dense(hidden, activation='relu', kernel_regularizer=l2(0.001)),
        Dense(n_steps_out)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

### Training Models

In [20]:
# Training univariate ARIMA
def train_ARIMA_uni(X_train, y_train, X_val, y_val, X_test, y_test):

    try:
        # Let target data be 1D
        if len(y_train.shape) > 1:
            y_train = y_train.ravel()
        if len(y_val.shape) > 1:
            y_val = y_val.ravel()
        if len(y_test.shape) > 1:
            y_test = y_test.ravel()

        print(f"    Tuning ARIMA params with AIC")
        fitted_model, best_params, best_aic = tune_ARIMA_params(y_train, exog_train=X_train)

        if fitted_model is None:
            print(f"    Tuning params failed")
            return None, None, None, None
        
        print(f"\n  Best ARIMA:{best_params}, AIC: {best_aic:.2f}")

        # Train again with Train+Val for Test
        y_trainval = np.concatenate([y_train, y_val])
        X_trainval = np.vstack([X_train, X_val])

        test_model = ARIMA(y_trainval, exog=X_trainval, order=best_params)
        fitted_test_model = test_model.fit()

        # For Multi-step ahead forecast
        if len(y_test) > 0 and len(X_test) > 0:
            steps_to_forecast = min(len(y_test), len(X_test))
            preds = fitted_test_model.forecast(steps=steps_to_forecast, exog=X_test[:steps_to_forecast])
    
        else:
            preds = np.array([])

        preds = np.array(preds).ravel()
        actual = y_test[:len(preds)]

        # Metrics
        if len(preds) > 0 and len(actual) > 0:
            rmse_score = rmse(preds, actual)
            mase_score = mase(actual, preds)
        else:
            rmse_score = np.inf
            mase_score = np.inf

        print(f"ARIMA - RMSE:{rmse_score:.4f}, MASE:{mase_score:.4f}")

        del fitted_test_model, test_model, fitted_model
        clear_memory()

        return preds, actual, rmse_score, mase_score
    
    except Exception as e:
        print(f"ARIMA failed {e}")
        return None, None, None, None

In [21]:
# Training DL models
def train_dl_model(model_build, X, y, years, model_name):
    try:
        print(f"Training {model_name}")
        
        # Prepare data for DL
        (X_train_dl, y_train_scaled, X_val_dl, y_val_scaled, X_test_dl, 
         y_test, y_scaler, test_years) = prepare_dl_data(X, y, years, N_STEPS_OUT, VAL_SIZE, TEST_SIZE)
        
        # Build model
        input_shape = (X_train_dl.shape[1], X_train_dl.shape[2])
        model = model_build(input_shape, N_STEPS_OUT)

        # Early stopping
        early_stopping = EarlyStopping(
            monitor='val_loss', patience=20, restore_best_weights=True, min_delta=0.001
        )

        # Set batch_size min as possible for runtime
        batch_size = min(8, len(X_train_dl))

        # Train model
        history = model.fit(
            X_train_dl, y_train_scaled,
            validation_data=(X_val_dl, y_val_scaled),
            epochs=50,
            batch_size=batch_size,
            callbacks=[early_stopping],
            verbose=0
        )

        preds_scaled = model.predict(X_test_dl, verbose=0)
        
        # Handle ED-LSTM 3D output (batch, steps, 1)
        if len(preds_scaled.shape) == 3:
            preds_scaled = preds_scaled.reshape(preds_scaled.shape[0], preds_scaled.shape[1])
        
        # Inverse transform
        preds_all = y_scaler.inverse_transform(preds_scaled)
        
        # Take only first pred from each multi-step forecast
        preds = preds_all[:, 0]
        actual = y_test[:, 0]
        
        # Metrics
        if len(preds) > 0 and len(actual) > 0:
            rmse_score = rmse(preds, actual)
            mase_score = mase(actual, preds)
        else:
            rmse_score = np.inf
            mase_score = np.inf
        
        print(f"{model_name} - RMSE:{rmse_score:.4f}, MASE:{mase_score:.4f}")
        
        del model, history
        clear_memory()
        
        return preds, actual, rmse_score, mase_score, test_years[:len(preds)]     

    except Exception as e:
        print(f"Traninig DL model failed {e}")
        return None, None, None, None, None

### Testing

In [22]:
# Testing univarite models
def test_uni_models(feature, target, lag_df, save_dir='data_export'):
    
    results = defaultdict(list)
    preds_data = {}

    for country in G20_COUNTRIES:
        country_data = lag_df[lag_df['country'] == country].sort_values('year')
        print(f"\nTesting for {country}")

        # Base data
        feature_cols = [feature]
        for i in range(1, N_STEPS_IN):
            lag_col = f"{feature}_lag{i}"
            if lag_col in country_data.columns:
                feature_cols.append(lag_col)
        
        all_cols = feature_cols + [target] + ['year']
        data_subset = country_data[all_cols].dropna()

        # Get years for plotting
        years = data_subset['year'].values

        # Data prep for ARIMA
        print(f"Training ARIMA")
        ARIMA_data = prepare_ARIMA_data(country_data, feature, target, N_STEPS_IN)
        
        X_train_ar, y_train_ar, X_val_ar, y_val_ar, X_test_ar, y_test_ar, test_years_ar = ARIMA_data

        # Test ARIMA
        preds, actual, rmse_score, mase_score = train_ARIMA_uni(
            X_train_ar, y_train_ar, X_val_ar, y_val_ar, X_test_ar, y_test_ar
            )
        
        if preds is not None:
            results['ARIMA'].append({
                'country': country,
                'rmse': rmse_score,
                'mase': mase_score
            })

            min_len = min(len(preds), len(actual), len(test_years_ar))
            preds_data[(country, 'ARIMA')] = {
                'predictions': preds[:min_len],
                'actuals': actual[:min_len],
                'years': test_years_ar[:min_len]
            }

        # Data prep for DL models
        X = data_subset[feature_cols].values
        y = data_subset[target].values

        model_configs = [
            ('LSTM', build_lstm),
            ('Bi-LSTM', build_bilstm),
            ('ED-LSTM', build_edlstm),
            ('CNN', build_cnn)
        ]

        for model_name, model_build in model_configs:
            result = train_dl_model(model_build, X, y, years, model_name)
            
            if result[0] is not None:
                preds, actual, rmse_score, mase_score, test_years_dl = result
                
                results[model_name].append({
                    'country': country,
                    'rmse': rmse_score,
                    'mase': mase_score
                })

                preds_data[(country, model_name)] = {
                    'predictions': preds,
                    'actuals': actual,
                    'years': test_years_dl
                }

    return results, preds_data

### Summary Result

In [23]:
def summary_result(all_results, save_dir='data_export'):

    summary_dir = os.path.join(save_dir, '02_results')
    if not os.path.exists(summary_dir):
        os.makedirs(summary_dir)

    summary_data = []

    for (feature, target), model_results in all_results.items():
        for model_name, results_list in model_results.items():
            rmse_values = [r['rmse'] for r in results_list]
            mase_values = [r['rmse'] for r in results_list]

            summary_data.append({
                'feature': feature,
                'target': target,
                'model': model_name,
                'n_countries': len(results_list),
                'avg_rmse': np.mean(rmse_values),
                'std_rmse': np.std(rmse_values),
                'min_rmse': np.min(rmse_values),
                'max_rmse': np.max(rmse_values),
                'avg_mase': np.mean(mase_values),
                'std_mase': np.std(mase_values),
                'min_mase': np.min(mase_values),
                'max_mase': np.max(mase_values)
            })

    summary_df = pd.DataFrame(summary_data)

    summary_df.to_csv(os.path.join(summary_dir, 'uni_model_summary.csv'), index=False)

    feature_rank = {}

    for target in TARGET_VARIABLES:
        target_df = summary_df[summary_df['target'] == target]

        feature_scores = target_df.groupby('feature')['avg_rmse'].mean().sort_values()
            
        feature_rank[target] = {
            'rankings': feature_scores.to_dict(),
            'ranked_features': feature_scores.index.tolist()
        }
        print(f"\nFeature Ranking for {target} (by average RMSE):")
        for i, (feature, score) in enumerate(feature_scores.items(), 1):
            print(f" {i}. {feature}: {score:.4f}")

    with open(os.path.join(summary_dir, 'univariate_feature_ranking.pkl'), 'wb') as f:
        pickle.dump(feature_rank, f)
    
    return summary_df, feature_rank

### Visualisation

In [24]:
def plot_preds(feature, target, preds_data, save_dir='data_export'):

    plot_dir = os.path.join(save_dir, '02_plots', '03_uni_models')
    if not os.path.exists(plot_dir):
        os.makedirs(plot_dir)

    countries = sorted(set([key[0] for key in preds_data.keys()]))
    models = ['ARIMA', 'LSTM', 'Bi-LSTM', 'ED-LSTM', 'CNN']

    for country in countries:
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        axes = axes.flatten()
        
        plot_idx = 0
        for model in models:
            if (country, model) in preds_data:
                data = preds_data[(country, model)]
                ax = axes[plot_idx]
                
                years = data['years']
                actual = data['actuals']
                preds = data['predictions']

                min_len = min(len(years), len(actual), len(preds))
                years = years[:min_len]
                actual = actual[:min_len]
                preds = preds[:min_len]

                # Plot actual vs predicted
                ax.plot(years, actual, 'b-', label='Actual', linewidth=2, marker='o')
                ax.plot(years, preds, 'r--', label='Predicted', linewidth=2, marker='s')
                
                # RMSE for title
                rmse_val = rmse(preds, actual)
                
                ax.set_title(f'{model}\nRMSE: {rmse_val:.4f}')
                ax.set_xlabel('Year')
                ax.set_ylabel(target)
                ax.legend()
                
                plot_idx += 1
        
        # Hide unused subplots
        for i in range(plot_idx, len(axes)):
            axes[i].set_visible(False)
        
        plt.suptitle(f'{country}: {feature} - {target}', fontsize=16)
        plt.tight_layout()
        
        # Save plot
        plot_file = os.path.join(plot_dir, f'{country}_{feature}_to_{target}.png')
        plt.savefig(plot_file, dpi=300, bbox_inches='tight')
        plt.close()
                
    print(f"Plots are saved")

In [ ]:
all_results = {}
all_preds_data = {}
target = 'co2'

for feature in tqdm(selected_features):
    if feature == target:
        continue

    print(f"Testing for feature: {feature}")

    results, preds_data = test_uni_models(feature, target, lag_df)

    if results:
        all_results[(feature, target)] = results
        for key, value in preds_data.items():
            all_preds_data[(feature, target, key[0], key[1])] = value

        # Results summary
        print(f"\nResults for {feature} to {target}")
        for model, model_results in results.items():
            if model_results:
                avg_rmse = np.mean([r['rmse'] for r in model_results])
                avg_mase = np.mean([r['mase'] for r in model_results])
                print(f"\n{model}: {len(model_results)} countries, avg RMSE: {avg_rmse:.4f}, avg MASE: {avg_mase:.4f}")

# Save all results
results_dir = os.path.join('data_export', '02_results')
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

with open(os.path.join(results_dir, f'{target}_univariate_predictions.pkl'), 'wb') as f:
    pickle.dump(all_preds_data, f)
print(f"All predictions saved to single file")

co2_summary_df, co2_feature_rank = summary_result(all_results)

  0%|          | 0/21 [00:00<?, ?it/s]

Testing for feature: electricity_generation

Testing for United States
Training ARIMA
ARIMA shape X:(35, 5), y:(35,)
Test data length: 6, Test years length: 6
    Tuning ARIMA params with AIC
    Not stationary p_value: 0.9894

  Best ARIMA:(3, 1, 2), AIC: 291.68
ARIMA - RMSE:198.0702, MASE:0.7647

Training LSTM
LSTM - RMSE:603.4701, MASE:2.3720
Training Bi-LSTM
Bi-LSTM - RMSE:646.9604, MASE:2.5637
Training ED-LSTM
ED-LSTM - RMSE:592.9695, MASE:2.3243
Training CNN
CNN - RMSE:593.8387, MASE:2.3312

Testing for China
Training ARIMA
ARIMA shape X:(35, 5), y:(35,)
Test data length: 6, Test years length: 6
    Tuning ARIMA params with AIC
    Not stationary p_value: 0.9890

  Best ARIMA:(0, 1, 1), AIC: 297.06
ARIMA - RMSE:1446.4328, MASE:3.8501
Training LSTM
LSTM - RMSE:1700.3673, MASE:5.0173
Training Bi-LSTM
Bi-LSTM - RMSE:3437.0344, MASE:9.9032
Training ED-LSTM
ED-LSTM - RMSE:5805.3019, MASE:17.1381
Training CNN
CNN - RMSE:533.9608, MASE:1.4797

Testing for Japan
Training ARIMA
ARIMA shap

  5%|▍         | 1/21 [28:09<9:23:13, 1689.69s/it]


Results for electricity_generation to co2

ARIMA: 19 countries, avg RMSE: 133.3861, avg MASE: 1.7073

LSTM: 19 countries, avg RMSE: 214.5092, avg MASE: 3.3994

Bi-LSTM: 19 countries, avg RMSE: 340.0546, avg MASE: 3.6976

ED-LSTM: 19 countries, avg RMSE: 438.8752, avg MASE: 4.6028

CNN: 19 countries, avg RMSE: 126.5099, avg MASE: 2.7243
Testing for feature: oil_consumption

Testing for United States
Training ARIMA
ARIMA shape X:(55, 5), y:(55,)
Test data length: 9, Test years length: 9
    Tuning ARIMA params with AIC
    Not stationary p_value: 0.8736

  Best ARIMA:(0, 1, 3), AIC: 434.97
ARIMA - RMSE:518.7587, MASE:2.5754
Training LSTM
LSTM - RMSE:254.0755, MASE:1.0993
Training Bi-LSTM
Bi-LSTM - RMSE:285.0589, MASE:1.0831
Training ED-LSTM
ED-LSTM - RMSE:257.8774, MASE:1.1183
Training CNN
CNN - RMSE:423.8124, MASE:1.7963

Testing for China
Training ARIMA
ARIMA shape X:(55, 5), y:(55,)
Test data length: 9, Test years length: 9
    Tuning ARIMA params with AIC
    Not stationary p_value:

 10%|▉         | 2/21 [57:10<9:04:30, 1719.51s/it]

...

 95%|█████████▌| 20/21 [10:41:54<45:22, 2722.43s/it] 


Results for energy_per_gdp to co2

ARIMA: 19 countries, avg RMSE: 91.5567, avg MASE: 2.0079

LSTM: 19 countries, avg RMSE: 707.4474, avg MASE: 12.2196

Bi-LSTM: 19 countries, avg RMSE: 691.8693, avg MASE: 13.0409

ED-LSTM: 19 countries, avg RMSE: 682.8813, avg MASE: 12.4230

CNN: 19 countries, avg RMSE: 709.6024, avg MASE: 12.7482
Testing for feature: population

Testing for United States
Training ARIMA
ARIMA shape X:(59, 5), y:(59,)
Test data length: 10, Test years length: 10
    Tuning ARIMA params with AIC
    Not stationary p_value: 0.5074

  Best ARIMA:(1, 1, 0), AIC: 501.82
ARIMA - RMSE:923.5988, MASE:4.0181
Training LSTM
LSTM - RMSE:676.7360, MASE:3.0504
Training Bi-LSTM
Bi-LSTM - RMSE:519.3137, MASE:2.1454
Training ED-LSTM
ED-LSTM - RMSE:366.3237, MASE:1.4654
Training CNN
CNN - RMSE:786.0012, MASE:3.8288

Testing for China
Training ARIMA
ARIMA shape X:(59, 5), y:(59,)
Test data length: 10, Test years length: 10
    Tuning ARIMA params with AIC
    Not stationary p_value: 1.000

100%|██████████| 21/21 [11:34:38<00:00, 1984.68s/it]


Results for population to co2

ARIMA: 19 countries, avg RMSE: 146.1398, avg MASE: 3.0689

LSTM: 19 countries, avg RMSE: 344.9429, avg MASE: 5.6610

Bi-LSTM: 19 countries, avg RMSE: 341.4722, avg MASE: 3.8913

ED-LSTM: 19 countries, avg RMSE: 308.8327, avg MASE: 6.2331

CNN: 19 countries, avg RMSE: 442.5368, avg MASE: 5.5545
All predictions saved to single file



Feature Ranking for co2 (by average RMSE):
 1. coal_consumption: 246.7179
 2. electricity_generation: 250.6670
 3. fossil_fuel_consumption: 264.8130
 4. oil_consumption: 274.1422
 5. population: 316.7849
 6. hydro_consumption: 338.2072
 7. energy_per_capita: 346.5072
 8. co2_per_unit_energy: 379.9888
 9. gas_prod_change_twh: 472.5887
 10. gas_consumption: 518.8964
 11. gas_cons_change_pct: 549.4952
 12. energy_cons_change_pct: 556.2726
 13. oil_cons_change_pct: 568.9613
 14. energy_per_gdp: 576.6714
 15. co2_growth_prct: 581.9248
 16. nuclear_consumption: 592.2019
 17. renewables_cons_change_twh: 646.7038
 18. low_carbon_consumption: 849.0293
 19. other_renewables_cons_change_twh: 959.2329
 20. renewables_consumption: 1047.6890
 21. wind_cons_change_twh: 2344.7404

Feature Ranking for gdp (by average RMSE):

Feature Ranking for primary_energy_consumption (by average RMSE):


In [24]:
all_results = {}
all_preds_data = {}
target = 'gdp'

for feature in tqdm(selected_features):
    if feature == target:
        continue

    print(f"Testing for feature: {feature}")

    results, preds_data = test_uni_models(feature, target, lag_df)

    if results:
        all_results[(feature, target)] = results
        for key, value in preds_data.items():
            all_preds_data[(feature, target, key[0], key[1])] = value

        # Results summary
        print(f"\nResults for {feature} to {target}")
        for model, model_results in results.items():
            if model_results:
                avg_rmse = np.mean([r['rmse'] for r in model_results])
                avg_mase = np.mean([r['mase'] for r in model_results])
                print(f"\n{model}: {len(model_results)} countries, avg RMSE: {avg_rmse:.4f}, avg MASE: {avg_mase:.4f}")

# Save all results
results_dir = os.path.join('data_export', '02_results')
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

with open(os.path.join(results_dir, f'{target}_univariate_predictions.pkl'), 'wb') as f:
    pickle.dump(all_preds_data, f)
print(f"All predictions saved to single file")

gdp_summary_df, gdp_feature_rank = summary_result(all_results)

  0%|          | 0/21 [00:00<?, ?it/s]

Testing for feature: electricity_generation

Testing for United States
Training ARIMA
ARIMA shape X:(34, 5), y:(34,)
Test data length: 6, Test years length: 6
    Tuning ARIMA params with AIC

  Best ARIMA:(2, 0, 1), AIC: 1293.04
ARIMA - RMSE:1453856343037.8840, MASE:2.2589

Training LSTM
LSTM - RMSE:2169343028945.2715, MASE:5.0042
Training Bi-LSTM
Bi-LSTM - RMSE:2186593930095.6909, MASE:5.0324
Training ED-LSTM
ED-LSTM - RMSE:3843837888426.6533, MASE:9.0029
Training CNN
CNN - RMSE:2382124554451.4019, MASE:5.5125

Testing for China
Training ARIMA
ARIMA shape X:(34, 5), y:(34,)
Test data length: 6, Test years length: 6
    Tuning ARIMA params with AIC
    Not stationary p_value: 1.0000

  Best ARIMA:(4, 1, 1), AIC: 1187.56
ARIMA - RMSE:749337380240.1266, MASE:0.5800
Training LSTM
LSTM - RMSE:4320610461052.1318, MASE:3.6715
Training Bi-LSTM
Bi-LSTM - RMSE:4971321881601.7061, MASE:4.2351
Training ED-LSTM
ED-LSTM - RMSE:303907177551.5201, MASE:0.2359
Training CNN
CNN - RMSE:453716010108.876

  5%|▍         | 1/21 [08:27<2:49:10, 507.50s/it]


Results for electricity_generation to gdp

ARIMA: 19 countries, avg RMSE: 239380788608.7308, avg MASE: 1.3540

LSTM: 19 countries, avg RMSE: 935295718961.7347, avg MASE: 7.1400

Bi-LSTM: 19 countries, avg RMSE: 892340209245.1260, avg MASE: 6.3879

ED-LSTM: 19 countries, avg RMSE: 1027505253554.6168, avg MASE: 10.1753

CNN: 19 countries, avg RMSE: 528717844913.1382, avg MASE: 6.3308
Testing for feature: oil_consumption

Testing for United States
Training ARIMA
ARIMA shape X:(54, 5), y:(54,)
Test data length: 9, Test years length: 9
    Tuning ARIMA params with AIC
    Not stationary p_value: 1.0000

  Best ARIMA:(1, 1, 1), AIC: 1958.34
ARIMA - RMSE:569962944619.0309, MASE:0.9260
Training LSTM
LSTM - RMSE:7051153512343.9141, MASE:17.3653
Training Bi-LSTM
Bi-LSTM - RMSE:7104276886412.8906, MASE:17.5056
Training ED-LSTM
ED-LSTM - RMSE:8099512483747.5234, MASE:19.9924
Training CNN
CNN - RMSE:8445726924256.2285, MASE:20.8684

Testing for China
Training ARIMA
ARIMA shape X:(54, 5), y:(54,)
T

 10%|▉         | 2/21 [19:01<3:04:14, 581.82s/it]

...

 95%|█████████▌| 20/21 [11:40:46<54:34, 3274.89s/it]  


Results for energy_per_gdp to gdp

ARIMA: 19 countries, avg RMSE: 494658249792.4243, avg MASE: 1.7386

LSTM: 19 countries, avg RMSE: 1989301094525.9490, avg MASE: 14.8228

Bi-LSTM: 19 countries, avg RMSE: 1912700107062.8059, avg MASE: 14.3746

ED-LSTM: 19 countries, avg RMSE: 2171303659109.7327, avg MASE: 14.7280

CNN: 19 countries, avg RMSE: 1845934594944.9236, avg MASE: 12.8120
Testing for feature: population

Testing for United States
Training ARIMA
ARIMA shape X:(58, 5), y:(58,)
Test data length: 10, Test years length: 10
    Tuning ARIMA params with AIC
    Not stationary p_value: 1.0000

  Best ARIMA:(2, 1, 2), AIC: 2126.70
ARIMA - RMSE:391808819344.5186, MASE:0.6718
Training LSTM
LSTM - RMSE:7150527067940.4746, MASE:16.9312
Training Bi-LSTM
Bi-LSTM - RMSE:2645165259953.9888, MASE:6.3365
Training ED-LSTM
ED-LSTM - RMSE:4766917843044.1426, MASE:10.9794
Training CNN
CNN - RMSE:539495191649.3331, MASE:1.2188

Testing for China
Training ARIMA
ARIMA shape X:(58, 5), y:(58,)
Test data

100%|██████████| 21/21 [12:44:10<00:00, 2183.37s/it]


Results for population to gdp

ARIMA: 19 countries, avg RMSE: 581761777244.5980, avg MASE: 2.1255

LSTM: 19 countries, avg RMSE: 1384913213499.5420, avg MASE: 9.1823

Bi-LSTM: 19 countries, avg RMSE: 1250530812086.5652, avg MASE: 8.8329

ED-LSTM: 19 countries, avg RMSE: 1755339670908.5271, avg MASE: 8.8346

CNN: 19 countries, avg RMSE: 1391827939381.0676, avg MASE: 10.8265


All predictions saved to single file

Feature Ranking for co2 (by average RMSE):

Feature Ranking for gdp (by average RMSE):
 1. electricity_generation: 724647963056.6693
 2. energy_per_capita: 1243996060032.9878
 3. population: 1272874682624.0601
 4. fossil_fuel_consumption: 1314068899495.4817
 5. oil_consumption: 1340722523711.3079
 6. coal_consumption: 1418808356663.6218
 7. nuclear_consumption: 1599552472202.6694
 8. hydro_consumption: 1614022131686.3579
 9. low_carbon_consumption: 1658183553979.1489
 10. energy_per_gdp: 1682779541087.1672
 11. gas_consumption: 1879474906826.1851
 12. gas_cons_change_pct: 1981452040558.1753
 13. renewables_cons_change_twh: 1983215483597.6392
 14. co2_per_unit_energy: 2075340617899.1394
 15. energy_cons_change_pct: 2080813034097.8601
 16. oil_cons_change_pct: 2106987891963.8250
 17. other_renewables_cons_change_twh: 2127132773624.6379
 18. co2_growth_prct: 2203831425901.6182
 19. gas_prod_change_twh: 2231279786835.3984
 20. renewables_consumption: 2

In [25]:
all_results = {}
all_preds_data = {}
target = 'primary_energy_consumption'

for feature in tqdm(selected_features):
    if feature == target:
        continue

    print(f"Testing for feature: {feature}")

    results, preds_data = test_uni_models(feature, target, lag_df)

    if results:
        all_results[(feature, target)] = results
        for key, value in preds_data.items():
            all_preds_data[(feature, target, key[0], key[1])] = value

        # Results summary
        print(f"\nResults for {feature} to {target}")
        for model, model_results in results.items():
            if model_results:
                avg_rmse = np.mean([r['rmse'] for r in model_results])
                avg_mase = np.mean([r['mase'] for r in model_results])
                print(f"\n{model}: {len(model_results)} countries, avg RMSE: {avg_rmse:.4f}, avg MASE: {avg_mase:.4f}")

# Save all results
results_dir = os.path.join('data_export', '02_results')
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

with open(os.path.join(results_dir, f'{target}_univariate_predictions.pkl'), 'wb') as f:
    pickle.dump(all_preds_data, f)
print(f"All predictions saved to single file")

energy_summary_df, energy_feature_rank = summary_result(all_results)

  0%|          | 0/21 [00:00<?, ?it/s]

Testing for feature: electricity_generation

Testing for United States
Training ARIMA
ARIMA shape X:(35, 5), y:(35,)
Test data length: 6, Test years length: 6
    Tuning ARIMA params with AIC
    Not stationary p_value: 0.3352

  Best ARIMA:(1, 1, 3), AIC: 346.60
ARIMA - RMSE:515.7260, MASE:0.4190
Training LSTM
LSTM - RMSE:750.8209, MASE:0.6854
Training Bi-LSTM
Bi-LSTM - RMSE:804.2878, MASE:0.7757
Training ED-LSTM
ED-LSTM - RMSE:798.4519, MASE:0.7672
Training CNN
CNN - RMSE:758.1251, MASE:0.6962

Testing for China
Training ARIMA
ARIMA shape X:(35, 5), y:(35,)
Test data length: 6, Test years length: 6
    Tuning ARIMA params with AIC
    Not stationary p_value: 1.0000

  Best ARIMA:(0, 1, 2), AIC: 330.67
ARIMA - RMSE:3068.0929, MASE:1.6180
Training LSTM
LSTM - RMSE:13241.1448, MASE:7.3681
Training Bi-LSTM
Bi-LSTM - RMSE:5879.9636, MASE:3.4022
Training ED-LSTM
ED-LSTM - RMSE:23062.0419, MASE:13.3498
Training CNN
CNN - RMSE:23632.3716, MASE:13.6784

Testing for Japan
Training ARIMA
ARIMA 

  5%|▍         | 1/21 [1:06:04<22:01:33, 3964.68s/it]


Results for electricity_generation to primary_energy_consumption

ARIMA: 19 countries, avg RMSE: 303.0429, avg MASE: 0.9123

LSTM: 19 countries, avg RMSE: 1298.3114, avg MASE: 4.1377

Bi-LSTM: 19 countries, avg RMSE: 930.5058, avg MASE: 4.7359

ED-LSTM: 19 countries, avg RMSE: 1832.4931, avg MASE: 5.0830

CNN: 19 countries, avg RMSE: 1778.9217, avg MASE: 3.6778
Testing for feature: oil_consumption

Testing for United States
Training ARIMA
ARIMA shape X:(55, 5), y:(55,)
Test data length: 9, Test years length: 9
    Tuning ARIMA params with AIC
    Not stationary p_value: 0.8239

  Best ARIMA:(1, 1, 1), AIC: 533.42
ARIMA - RMSE:916.8672, MASE:1.3007
Training LSTM
LSTM - RMSE:2605.8127, MASE:3.9722
Training Bi-LSTM
Bi-LSTM - RMSE:2380.2698, MASE:3.5631
Training ED-LSTM
ED-LSTM - RMSE:3178.8723, MASE:4.9378
Training CNN
CNN - RMSE:2393.4203, MASE:3.5178

Testing for China
Training ARIMA
ARIMA shape X:(55, 5), y:(55,)
Test data length: 9, Test years length: 9
    Tuning ARIMA params with A

 10%|▉         | 2/21 [2:33:21<24:52:22, 4712.76s/it]

...

 95%|█████████▌| 20/21 [32:34:04<1:45:16, 6316.23s/it]


Results for energy_per_gdp to primary_energy_consumption

ARIMA: 19 countries, avg RMSE: 297.0234, avg MASE: 1.8198

LSTM: 19 countries, avg RMSE: 2799.8270, avg MASE: 15.6358

Bi-LSTM: 19 countries, avg RMSE: 2734.1068, avg MASE: 14.4917

ED-LSTM: 19 countries, avg RMSE: 2775.8872, avg MASE: 14.8123

CNN: 19 countries, avg RMSE: 2614.8963, avg MASE: 13.8260
Testing for feature: population

Testing for United States
Training ARIMA
ARIMA shape X:(59, 5), y:(59,)
Test data length: 10, Test years length: 10
    Tuning ARIMA params with AIC
    Not stationary p_value: 0.4456

  Best ARIMA:(1, 1, 0), AIC: 611.70
ARIMA - RMSE:2856.0358, MASE:3.4319
Training LSTM
LSTM - RMSE:3071.4337, MASE:4.4376
Training Bi-LSTM
Bi-LSTM - RMSE:5240.5066, MASE:8.0453
Training ED-LSTM
ED-LSTM - RMSE:3860.8525, MASE:6.0384
Training CNN
CNN - RMSE:4964.9379, MASE:7.6955

Testing for China
Training ARIMA
ARIMA shape X:(59, 5), y:(59,)
Test data length: 10, Test years length: 10
    Tuning ARIMA params with AIC


100%|██████████| 21/21 [34:16:36<00:00, 5876.02s/it]  


Results for population to primary_energy_consumption

ARIMA: 19 countries, avg RMSE: 522.8253, avg MASE: 2.8542

LSTM: 19 countries, avg RMSE: 1487.0959, avg MASE: 5.4988

Bi-LSTM: 19 countries, avg RMSE: 1695.8028, avg MASE: 6.5220

ED-LSTM: 19 countries, avg RMSE: 1800.5593, avg MASE: 6.4649

CNN: 19 countries, avg RMSE: 1722.7945, avg MASE: 4.7623


All predictions saved to single file

Feature Ranking for co2 (by average RMSE):

Feature Ranking for gdp (by average RMSE):

Feature Ranking for primary_energy_consumption (by average RMSE):
 1. fossil_fuel_consumption: 778.4275
 2. energy_per_capita: 1193.1994
 3. electricity_generation: 1228.6550
 4. coal_consumption: 1363.1549
 5. oil_consumption: 1372.9593
 6. population: 1445.8156
 7. hydro_consumption: 1562.0440
 8. co2_per_unit_energy: 1758.7805
 9. nuclear_consumption: 1763.5545
 10. gas_consumption: 1938.8873
 11. gas_prod_change_twh: 2035.4358
 12. gas_cons_change_pct: 2190.9874
 13. energy_cons_change_pct: 2194.2085
 14. oil_cons_change_pct: 2238.5271
 15. energy_per_gdp: 2244.3481
 16. renewables_cons_change_twh: 2285.0570
 17. co2_growth_prct: 2354.7658
 18. low_carbon_consumption: 2384.5685
 19. other_renewables_cons_change_twh: 3050.7829
 20. renewables_consumption: 3704.0576
 21. wind_cons_change_twh: 6853.6169


In [ ]:
"""all_results = {}
all_preds_data = {}

for target in tqdm(TARGET_VARIABLES):
    print(f"\nTesting features for target {target}")
    for feature in selected_features:
        if feature == target:
            continue

        print(f"Testing for feature: {feature}")

        results, preds_data = test_uni_models(feature, target, lag_df)

        if results:
            all_results[(feature, target)] = results
            for key, value in preds_data.items():
                all_preds_data[(feature, target, key[0], key[1])] = value

            # Results summary
            print(f"\nResults for {feature} to {target}")
            for model, model_results in results.items():
                if model_results:
                    avg_rmse = np.mean([r['rmse'] for r in model_results])
                    avg_mase = np.mean([r['mase'] for r in model_results])
                    print(f"\n{model}: {len(model_results)} countries, avg RMSE: {avg_rmse:.4f}, avg MASE: {avg_mase:.4f}")

    # Save all results
    results_dir = os.path.join('data_export', '02_results')
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)

    with open(os.path.join(results_dir, '{target}_univariate_predictions.pkl'), 'wb') as f:
        pickle.dump(all_preds_data, f)
    print(f"All predictions saved to single file")

co2_summary_df, co2_feature_rank = summary_result(all_results)"""